In [2]:
import pandas as pd 
import matplotlib.pyplot as plt 
import pickle 
import numpy as np 
  
from sklearn.model_selection import train_test_split 
import sklearn.metrics as metrics 


In [3]:
import seaborn as sns

In [4]:
data2 = pd.read_csv('../Datasets/Cropdata.csv')

In [5]:
data3 = pd.read_csv('../Datasets/populate_data.csv')

In [212]:
data = data2.drop('Family' , axis = 'columns')

In [213]:
labels = data['label'].unique()

In [214]:


# Assuming 'df' is your DataFrame and 'class_labels_column' is the column containing class labels
# Step 1: Load your data
# df = pd.read_csv('your_dataset.csv')

# Step 2: Identify unique classes
labels= data['label'].unique()

# Step 3: Create new columns
for l in labels:
    data[l]= 0

# Step 4: Populate new columns
for index, row in data.iterrows():
    class_label = row['label']
    data.at[index, class_label] = 1

# Now, df contains one-hot encoded columns for your class labels


In [215]:
def semulate_yield(value):
    if value ==0:
        return np.random.uniform(0,0.5 ) if np.random.rand() <0.8 else np.random.uniform(0.5,1)
    else :
      return value

In [216]:
for column in labels:
    data[column]= data[column].apply(semulate_yield)

In [217]:
data = data.drop('label',axis ='columns')

In [16]:
exotic = pd.read_csv('../Datasets/imported_crops.csv')

In [20]:
exo_label = exotic['Name']

In [22]:
for l in exo_label:
    data[l]=0

C:\Users\Akshansh\AppData\Local\Temp\ipykernel_1788\387811044.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[l]=0
C:\Users\Akshansh\AppData\Local\Temp\ipykernel_1788\387811044.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[l]=0
C:\Users\Akshansh\AppData\Local\Temp\ipykernel_1788\387811044.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. T

In [31]:
df = {}

In [32]:
for l in labels:
    df[l] = data.loc[:,['N','P','K','temperature','humidity','ph','rainfall',l]]

In [36]:
def semulate_yield2(value):
    if value ==0:
        return 1 if np.random.rand() <0.011  else 0 
    else :
      return value

In [37]:
for l in exo_label:
    data[l] = data[l].apply(semulate_yield2)

In [39]:
for l in exo_label:
    data[l] = data[l].apply(semulate_yield)

In [43]:
exoticf ={}
for l in exo_label:
    exoticf[l]=data.loc[:,['N','P','K','temperature','humidity','ph','rainfall',l]]

In [48]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [178]:
class Predictor(nn.Module):
        def __init__(self):
            super(Predictor, self).__init__()
            self.fc = nn.Linear(7, 5) 
            self.fc1 = nn.Linear(5,1)
          # Example: 7 input features, 1 output node for regression

        def forward(self, x):
            x= self.fc(x)
            x = self.fc1(x)
         
            return x

In [195]:
modals = {}

In [78]:
class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels
    
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, idx):
        feature = torch.tensor(self.features.iloc[idx].values, dtype=torch.float32)
        output = torch.tensor(self.labels.iloc[idx], dtype=torch.float32)
        
        return feature, output


    


In [196]:
for l in labels:
    
    x = df[l].drop(l,axis ='columns')
    y = df[l][l]
    dataset = CustomDataset(x,y)
    nput_size = 7  # Example: Number of input features

    model = Predictor().to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters())

# Create your dataset and dataloader
    train_loader = DataLoader(dataset, batch_size=64, shuffle=True)
    for epoch in range(100):
    
     for inputs, values in train_loader:
         inputs = inputs.to(device)
         values = values.to(device)
        # Forward pass
         outputs = model(inputs)
         loss = criterion(outputs, values)
        
        # Calculate accuracy
        
      
        
        # Backward pass and optimization
         optimizer.zero_grad()
         loss.backward()
         optimizer.step()

    modals[l]=model
    

C:\Users\Akshansh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\Akshansh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([24])) that is different to the input size (torch.Size([24, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\Akshansh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\loc

In [229]:
for l in exo_label:
    
    x = exoticf[l].drop(l,axis ='columns')
    y = exoticf[l][l]
    dataset = CustomDataset(x,y)
    nput_size = 7  # Example: Number of input features

    model = Predictor().to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters())

# Create your dataset and dataloader
    train_loader = DataLoader(dataset, batch_size=64, shuffle=True)
    for epoch in range(100):
    
     for inputs, values in train_loader:
         inputs = inputs.to(device)
         values = values.to(device)
        # Forward pass
         outputs = model(inputs)
         loss = criterion(outputs, values)
        
        # Calculate accuracy
        
      
        
        # Backward pass and optimization
         optimizer.zero_grad()
         loss.backward()
         optimizer.step()

    modals[l]=model
    

C:\Users\Akshansh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\Akshansh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([24])) that is different to the input size (torch.Size([24, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\Akshansh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\loc

In [227]:
def recommender (input):
    for  l, m in 

N               24.000000
P              128.000000
K              196.000000
temperature     22.750888
humidity        90.694892
ph               5.521467
rainfall       110.431786
Vanilla          0.135697
Name: 1500, dtype: float64

In [172]:
test = torch.tensor(test,dtype=torch.float32)

C:\Users\Akshansh\AppData\Local\Temp\ipykernel_1788\2604536576.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  test = torch.tensor(test,dtype=torch.float32)


In [173]:
test

tensor([ 24.0000, 128.0000, 196.0000,  22.7509,  90.6949,   5.5215, 110.4318])

In [174]:
test =test.to(device)

In [203]:
for l,m in modals.items():
    m.eval()

In [204]:
output = {}

In [205]:
for l,m in modals.items():
    output[l]=m(test).item()

1178